In [ ]:
import numpy as np
import pandas as pd
from lazypredict.Supervised import LazyRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
import joblib

In [ ]:
paths=['/content/ResaleFlatPrices1.csv','/content/ResaleFlatPrices2.csv','/content/ResaleFlatPrices3.csv',
       '/content/ResaleFlatPrices4.csv','/content/Resaleflatprices5.csv']
dfs=[]
for i in paths:
  dff=pd.read_csv(i)
  dfs.append(dff)
df=pd.concat(dfs,ignore_index=True)

In [ ]:
df.drop(columns=['block','street_name','remaining_lease'],axis=1,inplace=True)

In [ ]:
df

,month,town,flat_type,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price
0,1990-01,ANG MO KIO,1 ROOM,10 TO 12,31.00,IMPROVED,1977,9000.00
1,1990-01,ANG MO KIO,1 ROOM,04 TO 06,31.00,IMPROVED,1977,6000.00
2,1990-01,ANG MO KIO,1 ROOM,10 TO 12,31.00,IMPROVED,1977,8000.00
3,1990-01,ANG MO KIO,1 ROOM,07 TO 09,31.00,IMPROVED,1977,6000.00
4,1990-01,ANG MO KIO,3 ROOM,04 TO 06,73.00,NEW GENERATION,1976,47200.00
...,...,...,...,...,...,...,...,...
911590,2023-11,YISHUN,4 ROOM,04 TO 06,104.00,Model A,1988,520000.00
911591,2023-11,YISHUN,5 ROOM,04 TO 06,112.00,Improved,2015,645000.00
911592,2023-11,YISHUN,5 ROOM,13 TO 15,113.00,Premium Apartment,2016,700000.00
911593,2023-11,YISHUN,5 ROOM,04 TO 06,121.00,Improved,1985,580000.00


In [ ]:
df[['year','month']]=df['month'].str.split('-',expand=True).astype(int)

In [ ]:
df[['storey_lower', 'storey_upper']] = df['storey_range'].str.split(' TO ', expand=True)
# Convert the lower and upper bounds to numeric values
df['storey_lower'] = pd.to_numeric(df['storey_lower'])
df['storey_upper'] = pd.to_numeric(df['storey_upper'])
df.drop('storey_range', axis=1, inplace=True)

In [ ]:
df['flat_type'].unique()

array(['1 ROOM', '3 ROOM', '4 ROOM', '5 ROOM', '2 ROOM', 'EXECUTIVE',
       'MULTI GENERATION', 'MULTI-GENERATION'], dtype=object)

In [ ]:
df['flat_type'] = df['flat_type'].str.replace('MULTI-GENERATION', 'MULTI GENERATION')

In [ ]:
df['flat_type'].unique()

array(['1 ROOM', '3 ROOM', '4 ROOM', '5 ROOM', '2 ROOM', 'EXECUTIVE',
       'MULTI GENERATION'], dtype=object)

In [ ]:
category_mapping = {
    '1 ROOM': 1,
    '2 ROOM': 2,
    '3 ROOM': 3,
    '4 ROOM': 4,
    '5 ROOM': 5,
    'EXECUTIVE': 6,
    'MULTI GENERATION': 7
}

df['flat_type'] = df['flat_type'].map(category_mapping)

df

,month,town,flat_type,floor_area_sqm,flat_model,lease_commence_date,resale_price,year,storey_lower,storey_upper
0,1,ANG MO KIO,1,31.00,IMPROVED,1977,9000.00,1990,10,12
1,1,ANG MO KIO,1,31.00,IMPROVED,1977,6000.00,1990,4,6
2,1,ANG MO KIO,1,31.00,IMPROVED,1977,8000.00,1990,10,12
3,1,ANG MO KIO,1,31.00,IMPROVED,1977,6000.00,1990,7,9
4,1,ANG MO KIO,3,73.00,NEW GENERATION,1976,47200.00,1990,4,6
...,...,...,...,...,...,...,...,...,...,...
911590,11,YISHUN,4,104.00,Model A,1988,520000.00,2023,4,6
911591,11,YISHUN,5,112.00,Improved,2015,645000.00,2023,4,6
911592,11,YISHUN,5,113.00,Premium Apartment,2016,700000.00,2023,13,15
911593,11,YISHUN,5,121.00,Improved,1985,580000.00,2023,4,6


In [ ]:
df['flat_model'].nunique()

34

In [ ]:
df['flat_model']=df['flat_model'].str.lower()

In [ ]:
df['flat_model'].nunique()

21

In [ ]:
flat_model_mapping = {
    'improved':1, 'new generation':2, 'model a':3, 'standard':4, 'simplified':5,
       'model a-maisonette':6, 'apartment':7, 'maisonette':8, 'terrace':9,
       '2-room':10, 'improved-maisonette':11, 'multi generation':12,
       'premium apartment':13, 'adjoined flat':14, 'premium maisonette':15,
       'model a2':16, 'dbss':17, 'type s1':18, 'type s2':19, 'premium apartment loft':20,
       '3gen':21
}

df['flat_model'] = df['flat_model'].map(flat_model_mapping)

df

,month,town,flat_type,floor_area_sqm,flat_model,lease_commence_date,resale_price,year,storey_lower,storey_upper
0,1,ANG MO KIO,1,31.00,1,1977,9000.00,1990,10,12
1,1,ANG MO KIO,1,31.00,1,1977,6000.00,1990,4,6
2,1,ANG MO KIO,1,31.00,1,1977,8000.00,1990,10,12
3,1,ANG MO KIO,1,31.00,1,1977,6000.00,1990,7,9
4,1,ANG MO KIO,3,73.00,2,1976,47200.00,1990,4,6
...,...,...,...,...,...,...,...,...,...,...
911590,11,YISHUN,4,104.00,3,1988,520000.00,2023,4,6
911591,11,YISHUN,5,112.00,1,2015,645000.00,2023,4,6
911592,11,YISHUN,5,113.00,13,2016,700000.00,2023,13,15
911593,11,YISHUN,5,121.00,1,1985,580000.00,2023,4,6


In [ ]:
df['town'].nunique()

27

In [ ]:
# Mapping of 'town' values to numbers
town_mapping = {
    'ANG MO KIO': 1,
    'BEDOK': 2,
    'BISHAN': 3,
    'BUKIT BATOK': 4,
    'BUKIT MERAH': 5,
    'BUKIT TIMAH': 6,
    'CENTRAL AREA': 7,
    'CHOA CHU KANG': 8,
    'CLEMENTI': 9,
    'GEYLANG': 10,
    'HOUGANG': 11,
    'JURONG EAST': 12,
    'JURONG WEST': 13,
    'KALLANG/WHAMPOA': 14,
    'MARINE PARADE': 15,
    'QUEENSTOWN': 16,
    'SENGKANG': 17,
    'SERANGOON': 18,
    'TAMPINES': 19,
    'TOA PAYOH': 20,
    'WOODLANDS': 21,
    'YISHUN': 22,
    'LIM CHU KANG': 23,
    'SEMBAWANG': 24,
    'BUKIT PANJANG': 25,
    'PASIR RIS': 26,
    'PUNGGOL': 27
}

df['town'] = df['town'].map(town_mapping)

df

,month,town,flat_type,floor_area_sqm,flat_model,lease_commence_date,resale_price,year,storey_lower,storey_upper
0,1,1,1,31.00,1,1977,9000.00,1990,10,12
1,1,1,1,31.00,1,1977,6000.00,1990,4,6
2,1,1,1,31.00,1,1977,8000.00,1990,10,12
3,1,1,1,31.00,1,1977,6000.00,1990,7,9
4,1,1,3,73.00,2,1976,47200.00,1990,4,6
...,...,...,...,...,...,...,...,...,...,...
911590,11,22,4,104.00,3,1988,520000.00,2023,4,6
911591,11,22,5,112.00,1,2015,645000.00,2023,4,6
911592,11,22,5,113.00,13,2016,700000.00,2023,13,15
911593,11,22,5,121.00,1,1985,580000.00,2023,4,6


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 911595 entries, 0 to 911594
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   month                911595 non-null  int64  
 1   town                 911595 non-null  int64  
 2   flat_type            911595 non-null  int64  
 3   floor_area_sqm       911595 non-null  float64
 4   flat_model           911595 non-null  int64  
 5   lease_commence_date  911595 non-null  int64  
 6   resale_price         911595 non-null  float64
 7   year                 911595 non-null  int64  
 8   storey_lower         911595 non-null  int64  
 9   storey_upper         911595 non-null  int64  
dtypes: float64(2), int64(8)
memory usage: 69.5 MB


In [ ]:
df.isna().sum()

month                  0
town                   0
flat_type              0
floor_area_sqm         0
flat_model             0
lease_commence_date    0
resale_price           0
year                   0
storey_lower           0
storey_upper           0
dtype: int64

In [ ]:
df

,month,town,flat_type,floor_area_sqm,flat_model,lease_commence_date,resale_price,year,storey_lower,storey_upper
0,1,1,1,31.00,1,1977,9000.00,1990,10,12
1,1,1,1,31.00,1,1977,6000.00,1990,4,6
2,1,1,1,31.00,1,1977,8000.00,1990,10,12
3,1,1,1,31.00,1,1977,6000.00,1990,7,9
4,1,1,3,73.00,2,1976,47200.00,1990,4,6
...,...,...,...,...,...,...,...,...,...,...
911590,11,22,4,104.00,3,1988,520000.00,2023,4,6
911591,11,22,5,112.00,1,2015,645000.00,2023,4,6
911592,11,22,5,113.00,13,2016,700000.00,2023,13,15
911593,11,22,5,121.00,1,1985,580000.00,2023,4,6


In [ ]:
dff=df.sample(n=10000, random_state=42)
dff

,month,town,flat_type,floor_area_sqm,flat_model,lease_commence_date,resale_price,year,storey_lower,storey_upper
134545,8,22,4,84.00,5,1988,268000.00,1996,4,6
607041,4,24,6,130.00,7,2001,425000.00,2010,1,3
309558,8,22,5,125.00,1,1985,321000.00,2000,1,3
741591,10,1,3,67.00,2,1978,301000.00,2016,4,6
473241,8,12,3,74.00,3,1983,168000.00,2005,1,3
...,...,...,...,...,...,...,...,...,...,...
624885,10,5,3,72.00,2,1983,344000.00,2010,1,3
338972,6,26,4,103.00,3,1996,278000.00,2001,16,18
881519,9,10,6,146.00,8,1986,798888.00,2022,7,9
785091,11,2,3,65.00,1,1982,270000.00,2018,7,9


In [ ]:
X=dff.drop('resale_price',axis=1)

In [ ]:
X

,month,town,flat_type,floor_area_sqm,flat_model,lease_commence_date,year,storey_lower,storey_upper
134545,8,22,4,84.00,5,1988,1996,4,6
607041,4,24,6,130.00,7,2001,2010,1,3
309558,8,22,5,125.00,1,1985,2000,1,3
741591,10,1,3,67.00,2,1978,2016,4,6
473241,8,12,3,74.00,3,1983,2005,1,3
...,...,...,...,...,...,...,...,...,...
624885,10,5,3,72.00,2,1983,2010,1,3
338972,6,26,4,103.00,3,1996,2001,16,18
881519,9,10,6,146.00,8,1986,2022,7,9
785091,11,2,3,65.00,1,1982,2018,7,9


In [ ]:
y=dff['resale_price']

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
reg=LazyRegressor(verbose=0,ignore_warnings=False,custom_metric=None)
models, predictions = reg.fit(x_train, x_test,y_train, y_test)
print(models)

 79%|███████▊  | 33/42 [01:02<00:08,  1.05it/s]

QuantileRegressor model failed to execute
Solver interior-point is not anymore available in SciPy >= 1.11.0.


100%|██████████| 42/42 [01:14<00:00,  1.78s/it]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001013 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 317
[LightGBM] [Info] Number of data points in the train set: 8000, number of used features: 9
[LightGBM] [Info] Start training from score 317644.048984
                               Adjusted R-Squared  R-Squared      RMSE  \
Model                                                                    
XGBRegressor                                 0.96       0.96  33867.12   
HistGradientBoostingRegressor                0.96       0.96  35662.71   
LGBMRegressor                                0.95       0.95  35798.53   
RandomForestRegressor                        0.93       0.93  45789.09   
BaggingRegressor                             0.92       0.92  47200.52   
GradientBoostingRegressor                    0.92       0.92  47494.76   
ExtraTreesRegressor                          0.92       0.92  48

In [ ]:
rfr=RandomForestRegressor(max_depth=20,max_features= 'sqrt', min_samples_leaf= 1, min_samples_split= 10, n_estimators= 200)
rfr.fit(x_train,y_train)
y_pred=rfr.predict(x_test)
mse=mean_squared_error(y_test,y_pred)
mae=mean_absolute_error(y_test,y_pred)
rmse=np.sqrt(mse)
r2=r2_score(y_test,y_pred)
print("Mean Absolute Error (MAE)    :", mae)
print("Mean Squared Error (MSE)     :", mse)
print("Root Mean Square Error (RMSE):", rmse)
print("R-squared (R2) Score         :", r2)

Mean Absolute Error (MAE)    : 32190.946973718263
Mean Squared Error (MSE)     : 2279025177.54791
Root Mean Square Error (RMSE): 47739.13674908576
R-squared (R2) Score         : 0.9198280248696211


In [ ]:
joblib.dump(rfr,'real_estate.joblib')

['real_estate.joblib']